#Data Loading

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os

In [ ]:
#Carga de datos
os.chdir("/content/drive/MyDrive/TAC Notebooks/TA Final/archive/")

In [ ]:
#TODO:
#Dataframe: Concat week text | Current week open | Next week open | Absolute change | Relative change | Type of change (Positive, Negative)

#carga dataset tweets
used_cols = ["body","ticker_symbol","current_monday"]
df_company = pd.read_csv("Company.csv")
df_company_tweet = pd.read_csv("Company_Tweet.csv")
df_tweet = pd.read_csv("Tweet.csv")
df_tweet.post_date = df_tweet.post_date.apply(datetime.fromtimestamp)
#Juntamos los dataset
df_tweets = pd.merge(df_tweet,df_company_tweet,on='tweet_id',how='left')
#Finding corresponding monday to each tweet
df_tweets["current_monday"] = df_tweets.post_date.apply(lambda x:(x - timedelta(days = x.weekday())).replace(hour=0, minute=0, second=0))
df_tweets = df_tweets[used_cols]
#carga de datos acciones
used_cols = ["Date","Open"]
df_stocks = None
for ticker_symbol in df_company.ticker_symbol:
  df_ticker_stocks = pd.read_csv(ticker_symbol+".csv")
  df_ticker_stocks["ticker_symbol"] = ticker_symbol
  if df_stocks is None: df_stocks = df_ticker_stocks
  else: df_stocks = pd.concat([df_stocks,df_ticker_stocks])
df_stocks.Date = df_stocks.Date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
#Procesamiento de data financiera
df_stocks["next_monday"] = df_stocks.Date.apply(lambda x:x + timedelta(days = 7))
df_stocks = pd.merge(df_stocks,df_stocks.drop(columns="next_monday"),left_on=["ticker_symbol","next_monday"],right_on=["ticker_symbol","Date"],how='left')
df_stocks.dropna(inplace=True)
df_stocks.drop(columns="Date_y",inplace=True)
df_stocks.rename(columns={'Date_x': 'Date'}, inplace=True)
df_stocks["abs_dif"] = df_stocks.Open_y - df_stocks.Open_x
df_stocks["rel_dif"] = (df_stocks.Open_y - df_stocks.Open_x) / df_stocks.Open_x
df_stocks["type_dif"] = df_stocks.abs_dif.apply(lambda x: 0 if x==0 else x/abs(x)).astype("int32")
df_stocks.rename(columns={"Date":"current_monday"},inplace=True)
#Concatenamos tweets por dia y por empresa
df_tweets = df_tweets.groupby(["ticker_symbol","current_monday"]).body.apply('. '.join).reset_index()
#Combinamos los dataset
df = pd.merge(df_tweets,df_stocks,on=["current_monday","ticker_symbol"],how="left")

In [ ]:
df.head()

,ticker_symbol,current_monday,body,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,next_monday,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,abs_dif,rel_dif,type_dif
0,AAPL,2014-12-29,"lx21 made $10,008 on $AAPL -Check it out! htt...",28.447500,28.692499,26.837500,27.332500,24.819241,608353600,2015-01-05,27.072500,28.312500,26.157499,28.002501,25.427635,1.133010e+09,-1.375000,-0.048335,-1
1,AAPL,2015-01-05,Stock Contest!! Pick $AAPL and WIN a FREE Tabl...,27.072500,28.312500,26.157499,28.002501,25.427635,1133010000,2015-01-12,28.150000,28.200001,26.299999,26.497499,24.061016,1.216906e+09,1.077500,0.039801,1
2,AAPL,2015-01-12,#TOPTICKERTWEETS $FB $GOOG $GOOGL $YHOO $SPY $...,28.150000,28.200001,26.299999,26.497499,24.061016,1216906400,2015-01-19,26.959999,28.437500,26.625000,28.245001,25.647839,7.949480e+08,-1.190001,-0.042274,-1
3,AAPL,2015-01-19,"THE BINARY OPTIONS COMPETITION WIN $100,000 ht...",26.959999,28.437500,26.625000,28.245001,25.647839,794948000,2015-01-26,28.434999,30.000000,27.257500,29.290001,26.596750,1.863371e+09,1.475000,0.054711,1
4,AAPL,2015-01-26,If Apple reports it sold over 70M iPhones in t...,28.434999,30.000000,27.257500,29.290001,26.596750,1863370800,2015-02-02,29.512501,30.127501,29.020000,29.732500,26.998556,1.083029e+09,1.077502,0.037894,1


In [ ]:
df.shape

(1572, 19)

In [ ]:
#Guardar dataset
df.to_csv("concat_dataset.csv",index=False)

In [ ]:
#Dataset de registros no concatenados
df.head()

,body,ticker_symbol,current_monday,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,next_monday,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,abs_dif,rel_dif,type_dif
0,"lx21 made $10,008 on $AAPL -Check it out! htt...",AAPL,2014-12-29,28.447500,28.692499,26.837500,27.332500,24.819241,608353600,2015-01-05,27.07250,28.312500,26.157499,28.002501,25.427635,1.133010e+09,-1.375000,-0.048335,-1
1,Insanity of today weirdo massive selling. $aap...,AAPL,2014-12-29,28.447500,28.692499,26.837500,27.332500,24.819241,608353600,2015-01-05,27.07250,28.312500,26.157499,28.002501,25.427635,1.133010e+09,-1.375000,-0.048335,-1
2,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,AMZN,2014-12-29,307.850006,314.750000,306.579987,308.519989,308.519989,9933200,2015-01-05,307.01001,308.380005,292.380005,296.929993,296.929993,1.461430e+07,-0.839996,-0.002729,-1
3,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,TSLA,2014-12-29,45.380001,45.582001,42.652000,43.862000,43.862000,63838000,2015-01-05,42.91000,43.299999,40.841999,41.332001,41.332001,1.135480e+08,-2.470001,-0.054429,-1
4,Swing Trading: Up To 8.91% Return In 14 Days h...,AAPL,2014-12-29,28.447500,28.692499,26.837500,27.332500,24.819241,608353600,2015-01-05,27.07250,28.312500,26.157499,28.002501,25.427635,1.133010e+09,-1.375000,-0.048335,-1


In [ ]:
df.shape

(4336445, 19)

In [ ]:
#Guardamos dataset sin concatenar tweets
df.to_csv("non_concat_dataset.csv",index=False)

#Data Preprocessing

In [ ]:
pip install -q tf-models-official==2.4.0

     |████████████████████████████████| 1.1MB 5.1MB/s 
     |████████████████████████████████| 38.2MB 113kB/s 
     |████████████████████████████████| 174kB 47.1MB/s 
     |████████████████████████████████| 358kB 45.3MB/s 
     |████████████████████████████████| 1.2MB 38.9MB/s 
     |████████████████████████████████| 645kB 32.5MB/s 
     |████████████████████████████████| 686kB 35.4MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

#submodulos requeridos para prueba
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

In [ ]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

In [ ]:
glue, info = tfds.load('glue/mrpc', with_info=True,
                       batch_size=-1)